library

In [ ]:
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
from pandas import datetime
import math, time
import itertools
from sklearn import preprocessing
#from sklearn.preprocessing import MinMaxScaler
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Price.csv")
df

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0
...,...,...,...,...,...,...,...
851259,2016-12-30,ZBH,103.309998,103.199997,102.849998,103.930000,973800.0
851260,2016-12-30,ZION,43.070000,43.040001,42.689999,43.310001,1938100.0
851261,2016-12-30,ZTS,53.639999,53.529999,53.270000,53.740002,1701200.0
851262,2016-12-30,AIV,44.730000,45.450001,44.410000,45.590000,1380900.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    851264 non-null  object 
 1   symbol  851264 non-null  object 
 2   open    851264 non-null  float64
 3   close   851264 non-null  float64
 4   low     851264 non-null  float64
 5   high    851264 non-null  float64
 6   volume  851264 non-null  float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


PreProcess

In [ ]:
stock_df = df[df.symbol == 'YHOO'] #choose one stock
print(stock_df)

              date symbol       open  ...        low       high      volume
713     2010-01-04   YHOO  16.940001  ...  16.879999  17.200001  16587400.0
1181    2010-01-05   YHOO  17.219999  ...  17.000000  17.230000  11718100.0
1649    2010-01-06   YHOO  17.170000  ...  17.070000  17.299999  16422000.0
2117    2010-01-07   YHOO  16.809999  ...  16.570000  16.900000  31816300.0
2585    2010-01-08   YHOO  16.680000  ...  16.620001  16.760000  15470000.0
...            ...    ...        ...  ...        ...        ...         ...
849257  2016-12-23   YHOO  38.459999  ...  38.369999  38.810001   3840100.0
849757  2016-12-27   YHOO  38.590000  ...  38.500000  39.070000   4240300.0
850257  2016-12-28   YHOO  39.119999  ...  38.709999  39.220001   4393500.0
850757  2016-12-29   YHOO  38.759998  ...  38.480000  38.930000   4170200.0
851257  2016-12-30   YHOO  38.720001  ...  38.430000  39.000000   6431600.0

[1762 rows x 7 columns]


In [ ]:
stock_df.tail()

,date,symbol,open,close,low,high,volume
849257,2016-12-23,YHOO,38.459999,38.660000,38.369999,38.810001,3840100.0
849757,2016-12-27,YHOO,38.590000,38.919998,38.500000,39.070000,4240300.0
850257,2016-12-28,YHOO,39.119999,38.730000,38.709999,39.220001,4393500.0
850757,2016-12-29,YHOO,38.759998,38.639999,38.480000,38.930000,4170200.0
851257,2016-12-30,YHOO,38.720001,38.669998,38.430000,39.000000,6431600.0


In [ ]:
stock_df_dates = pd.to_datetime(df['date'])
print(stock_df_dates)


0        2016-01-05
1        2016-01-06
2        2016-01-07
3        2016-01-08
4        2016-01-11
            ...    
851259   2016-12-30
851260   2016-12-30
851261   2016-12-30
851262   2016-12-30
851263   2016-12-30
Name: date, Length: 851264, dtype: datetime64[ns]


In [ ]:
cols = list(stock_df)[2:7]
print(cols)
df_training = stock_df[cols].astype(float)
print(df_training)

['open', 'close', 'low', 'high', 'volume']
             open      close        low       high      volume
713     16.940001  17.100000  16.879999  17.200001  16587400.0
1181    17.219999  17.230000  17.000000  17.230000  11718100.0
1649    17.170000  17.170000  17.070000  17.299999  16422000.0
2117    16.809999  16.700001  16.570000  16.900000  31816300.0
2585    16.680000  16.700001  16.620001  16.760000  15470000.0
...           ...        ...        ...        ...         ...
849257  38.459999  38.660000  38.369999  38.810001   3840100.0
849757  38.590000  38.919998  38.500000  39.070000   4240300.0
850257  39.119999  38.730000  38.709999  39.220001   4393500.0
850757  38.759998  38.639999  38.480000  38.930000   4170200.0
851257  38.720001  38.669998  38.430000  39.000000   6431600.0

[1762 rows x 5 columns]


In [ ]:
scaler = StandardScaler()
scaler = scaler.fit(df_training)
df_training_scaled = scaler.transform(df_training)

df_training_scaled

array([[-0.9078427 , -0.89375147, -0.89448224, -0.90342189, -0.24567839],
       [-0.88330165, -0.88235157, -0.88385001, -0.90081845, -0.59943356],
       [-0.88768392, -0.88761306, -0.87764793, -0.89474364, -0.25769472],
       ...,
       [ 1.03617278,  1.00301653,  1.03968139,  1.00756709, -1.13156654],
       [ 1.00461969,  0.9951242 ,  1.01930321,  0.98239957, -1.14778931],
       [ 1.00111406,  0.99775486,  1.01487315,  0.98847447, -0.98349836]])

In [ ]:
#splitting
train_size = int(len(df_training_scaled)*.8)
test_size = len(df_training_scaled)-train_size
train_data = df_training_scaled[0:train_size,:]
test_data = df_training_scaled[train_size:len(df_training_scaled),:] 


In [ ]:
#data for lstm
n_future = 1
n_past = 14

trainx = []
trainy = []

for i in range(n_past, len(train_data) - n_future +1):
   trainx.append(train_data[i - n_past:i, 0:train_data.shape[1]])
   trainy.append(train_data[i + n_future-1:i + n_future, 0])
  
trainX , trainY = np.array(trainx) , np.array(trainy)

testx = []
testy = []

for i in range(n_past, len(test_data) - n_future +1):
   testx.append(test_data[i - n_past:i, 0:train_data.shape[1]])
   testy.append(test_data[i + n_future-1:i + n_future, 0])
  
testX , testY = np.array(testx) , np.array(testy)

print("trainX shape == {}.".format(trainX.shape))
print("trainY shape == {}.".format(trainY.shape))
print("testX shape == {}.".format(testX.shape))
print("testY shape == {}.".format(testY.shape))


trainX shape == (1395, 14, 5).
trainY shape == (1395, 1).
testX shape == (339, 14, 5).
testY shape == (339, 1).


Model

In [161]:
# Hyper-parameters 
input_size = 5
sequence_length = 14
hidden_size = 20
num_classes = 2
num_epochs = 10
batch_size = 16
learning_rate = 0.001
num_layers = 1
num_timesteps = 14

In [ ]:
# LSTM
#Building the Model
class MV_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MV_LSTM, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers 
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) 
        #(batch, seq, input)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size*sequence_length, 1)

    def init_hidden(self, batch_size):
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size) 
        self.hidden = (h0,c0)
   
    def forward(self, x):
        batch_size, sequence_length, _ = x.size()
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        x = lstm_out.contiguous().view(batch_size, -1)
        out = self.fc(x)
        return out

#model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

In [ ]:
#create
mv_model = MV_LSTM(input_size,num_timesteps)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(mv_model.parameters(), lr=learning_rate)  
print(mv_model)

MV_LSTM(
  (lstm): LSTM(5, 14, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=196, out_features=1, bias=True)
)


In [162]:
#Training
train_episodes = 100
mv_model.train()
for t in range(train_episodes):
    for b in range(0,len(trainX),batch_size):
        inpt = trainX[b:b+batch_size,:,:]
        target =trainY[b:b+batch_size]    
        
        x_batch = torch.tensor(inpt,dtype=torch.float32)    
        y_batch = torch.tensor(target,dtype=torch.float32)
       
    
        mv_model.init_hidden(x_batch.size(0))
    #    lstm_out, _ = mv_net.l_lstm(x_batch,nnet.hidden)    
    #    lstm_out.contiguous().view(x_batch.size(0),-1)
        output = mv_model(x_batch) 
        loss = criterion(output.view(-1), y_batch)
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()   


    if t % 10 == 0 and t !=0:
      print('epoch : ' , t , 'loss : ' , loss.item())


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([16, 1])) that is different to the input size (torch.Size([16])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([3, 1])) that is different to the input size (torch.Size([3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch :  10 loss :  0.0014428147114813328
epoch :  20 loss :  0.0013486738316714764
epoch :  30 loss :  0.0013928315602242947
epoch :  40 loss :  0.001444479450583458
epoch :  50 loss :  0.0015163564821705222
epoch :  60 loss :  0.0014953078934922814
epoch :  70 loss :  0.0014026120770722628
epoch :  80 loss :  0.0013727897312492132
epoch :  90 loss :  0.0016873758286237717


In [177]:
 y_batch

tensor([[0.8784],
        [0.8802],
        [0.8083]])

In [176]:
#test model
n_correct = 0
n_samples = 0
test_episodes = 100
with torch.no_grad():
  for t in range(test_episodes):
    for b in range(0,len(testX),batch_size):
        inpt_t = testX[b:b+batch_size,:,:]
        target_t =testY[b:b+batch_size]    
        mv_model.eval()
        x_batch_t = torch.tensor(inpt,dtype=torch.float32)    
        y_batch_t = torch.tensor(target,dtype=torch.float32)
        output = mv_model(x_batch_t)
        loss = criterion(output.view(-1), y_batch_t) 
    if t % 10 == 0 and t !=0:
      print('epoch : ' , t , 'loss : ' , loss.item())




#acc = 100.0 * n_correct / n_samples
#print('Accuracy : {acc} %')
#print(predicted) 


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([3, 1])) that is different to the input size (torch.Size([3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch :  10 loss :  0.001446894253604114
epoch :  20 loss :  0.001446894253604114
epoch :  30 loss :  0.001446894253604114
epoch :  40 loss :  0.001446894253604114
epoch :  50 loss :  0.001446894253604114
epoch :  60 loss :  0.001446894253604114
epoch :  70 loss :  0.001446894253604114
epoch :  80 loss :  0.001446894253604114
epoch :  90 loss :  0.001446894253604114


In [172]:
output.view(-1)

tensor([0.8821, 0.8687, 0.8453])